<a href="https://colab.research.google.com/github/guptaaryanr/btc-bots/blob/main/bitcoin_trading_botv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install websocket-client

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
import websocket
import numpy as np
import json
from talib import abstract
import inspect

In [ ]:
symbol = 'btcusdt'
interval = '1m'
socket = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'

In [ ]:
amount = 1000
core_amount = amount * 0.75
core_quantity = 0
trade_amount = amount * 0.25
core_to_trade = True
transaction_cost = 0.0005
min_trade_amt = 30
portfolio = 0
avg_indicator, amt, trade_amt, port_value = 0, 0, 0, 0
investment = []
real_time_portfolio_value = []
closes, highs, lows, opens, volumes = [], [], [], [], []
indicators = []
money_end = amount
candles = [opens, highs, lows, closes, volumes]

In [ ]:
def buy(allocated_money, price):
    global portfolio, money_end, investment
    quantity = allocated_money / price
    money_end -= (quantity * price)
    portfolio += quantity
    if investment == []:
        investment.append(allocated_money)
    else:
        investment.append(allocated_money)
        investment[-1] += investment[-2]
        
def sell(allocated_money, price):
    global portfolio, money_end, investment
    quantity = allocated_money / price
    money_end += (quantity * price)
    portfolio -= quantity
    investment.append(-allocated_money)
    investment[-1] += investment[-2]

In [ ]:
f = abstract
dir1 = dir(f)
public_method_names = [method for method in dir1 if method.startswith('CDL')]

In [ ]:
def on_message(ws, message):
    global portfolio, investment, money_end, real_time_portfolio_value
    global highs, lows, closes, core_to_trade, core_quantity
    json_message = json.loads(message)
    cs = json_message['k']
    candle_closed = cs['x']
    close = cs['c']
    high = cs['h']
    low = cs['l']
    open = cs['o']
    volume = cs['v']
    candle = [open, high, low, close, volume]

    if candle_closed:
        for i in candles:
            i.append(float(candle[candles.index(i)]))
        print(f'Closes: {closes}')

        inputs = {'open': np.array(opens), 'high': np.array(highs), 'low': np.array(lows), 'close': np.array(closes), 'volume': np.array(volumes)}
        print(inputs)

        if core_to_trade:
            buy(core_amount, closes[-1])
            print(f'Core Investment: Bought ${core_amount} BTC')
            core_quantity += core_amount / closes[-1]
            core_to_trade = False

        for method in public_method_names:
            indicator = getattr(f, method)(inputs)
            indicators.append(indicator[-1])
        avg_indicators = np.mean(indicators)

        if avg_indicators >= 10:
            amt = trade_amount
        elif avg_indicators <= -10:
            amt = -trade_amount
        else:
            amt = avg_indicators * 10

        port_value = portfolio * closes[-1] - core_quantity * closes[-1]
        trade_amt = amt - port_value
        real_time_portfolio_value.append(float(money_end + portfolio * closes[-1]))
        print(f'Average of all indicators is {avg_indicators} and recommended exposure is ${amt}')
        print(f'Real-Time Portfolio Value: ${real_time_portfolio_value[-1]}')
        print(f'Invested amount: ${portfolio * closes[-1]}')

        if trade_amt > min_trade_amt:
            buy(trade_amt, closes[-1])
            print(f'Bought ${trade_amt} BTC')
        elif trade_amt < -min_trade_amt:
            sell(-trade_amt, closes[-1])
            print(f'Sold ${trade_amt} BTC')

        print('\n')

ws = websocket.WebSocketApp(socket, on_message = on_message)

In [ ]:
ws.run_forever()

In [ ]:
investment

In [ ]:
port_value = portfolio * closes[-1]
if port_value > 0:
    sell(port_value, closes[-1])
else:
    buy(-port_value, closes[-1])
money_end += investment[-1]
print('All trades settled')

In [ ]:
money_end

In [ ]:
portfolio

In [ ]:
amount

In [ ]:
btc_return = closes[-1] / closes[0] - 1
bot_return = real_time_portfolio_value[-1] / amount - 1
alpha = bot_return - btc_return
btc_risk = np.std(np.log(np.array(closes[1:]) / np.array(closes[:-1])))
bot_risk = np.std(np.log(np.array(real_time_portfolio_value[1:]) / np.array(real_time_portfolio_value[:-1])))
btc_sharpe = round(btc_return / bot_risk, 3)
bot_sharpe = round(bot_return / bot_risk, 3)
print(btc_return, ',',  btc_risk, ',', btc_sharpe)
print(bot_return, ',', bot_risk, ',', bot_sharpe)
print(alpha)